<a href="https://colab.research.google.com/github/ucaokylong/LLM_learning/blob/main/L3_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
openai_api_key = ""

## Cài các thư viện liên quan

In [ ]:
!pip install --quiet -U langchain chromadb langchain-openai pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
[]

## Tóm tắt

In [ ]:
def split_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Remove leading/trailing whitespace and split the content into sentences
    sentences = content.strip().split('.')

    # Remove empty sentences
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    # Calculate the number of sentences per chunk
    total_sentences = len(sentences)
    chunk_size = total_sentences // 10

    chunks = []
    start = 0
    for i in range(9):
        end = start + chunk_size
        chunk = sentences[start:end]
        chunks.append('. '.join(chunk) + '.')
        start = end

    # Add the remaining sentences to the last chunk
    last_chunk = sentences[start:]
    chunks.append('. '.join(last_chunk) + '.')

    return chunks

# Example usage
file_path = '/content/vinallama.txt'
chunks = split_file(file_path)

for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i}:")
    print(chunk)
    print()

Chunk 1:
In this technical report, we present VinaLLaMA, an open-source, state-of-the-art (SOTA) Large Language Model for the Vietnamese language, built upon LLaMA-2 with an additional 800 billion trained tokens. VinaLLaMA not only demonstrates fluency in Vietnamese but also exhibits a profound understanding of Vietnamese culture, making it a truly indigenous model. VinaLLaMA-7B-chat, trained on 1-million high quality synthetic samples, achieves SOTA results on key benchmarks, including VLSP, VMLU, and Vicuna Benchmark Vietnamese, marking a significant advancement in the Vietnamese AI landscape and offering a versatile resource for various applications. 1 Introduction 
The surge in Large Language Models (LLMs) such as ChatGPT and GPT-4 has significantly advanced the field of artificial intelligence (AI), particularly in language processing. In 2023, Vietnam’s AI sector witnessed a notable development with the introduction of several Vietnamese-centric LLMs, including BLOOMZ’s Vietcuna,

Summarization (Tóm Tắt) là một trong số những task khó nhất cho các LLM. Nó đỏi hỏi LLM có khả năng xử lí long context nhưng vẫn có khả năng làm retrieval trên text tốt. Vì vậy, đối vơi đại đa số các LLM, chúng ta sẽ làm prompt chaining để triển khai bài toán này

In [ ]:
question = """
Hãy tóm tắt văn bản: {context}
"""

summarize_prompt = """
Sau đây là một nội dung được trích xuất của một văn bản, nhiệm vụ của bạn là hãy tóm tắt nó
---CONTEXT---
{context}
---END CONTEXT---
Hãy đưa ra tóm tắt của văn bản trên. Tôi chỉ cần phần tóm tắt, và không cần thêm bất kì thứ gì khác.
Tóm Tắt:
"""

final_ans_prompt = """
Dưới đây là các đoạn tóm tắt của từng đoạn nhỏ của một văn bản lớn.
---CONTEXT---
{context}
---END CONTEXT---
Dựa trên các đoạn trên, hãy đưa ra tóm tắt tổng của tất cả các đoạn trên. Tôi chỉ cần tóm tắt tổng, không cần thêm bất kì thứ gì khác.
Tóm tắt tổng:
"""

In [ ]:
def build_final_context(chunks):
    context = ""
    for index, chunk in enumerate(chunks):
        context +=  f"Context {index + 1}: " + chunk + "\n"
    return context

In [ ]:
build_final_context(chunks)

'Context 1: In this technical report, we present VinaLLaMA, an open-source, state-of-the-art (SOTA) Large Language Model for the Vietnamese language, built upon LLaMA-2 with an additional 800 billion trained tokens. VinaLLaMA not only demonstrates fluency in Vietnamese but also exhibits a profound understanding of Vietnamese culture, making it a truly indigenous model. VinaLLaMA-7B-chat, trained on 1-million high quality synthetic samples, achieves SOTA results on key benchmarks, including VLSP, VMLU, and Vicuna Benchmark Vietnamese, marking a significant advancement in the Vietnamese AI landscape and offering a versatile resource for various applications. 1 Introduction \nThe surge in Large Language Models (LLMs) such as ChatGPT and GPT-4 has significantly advanced the field of artificial intelligence (AI), particularly in language processing. In 2023, Vietnam’s AI sector witnessed a notable development with the introduction of several Vietnamese-centric LLMs, including BLOOMZ’s Vietc

So sánh câu trả lời

In [ ]:
from openai import OpenAI

llm = OpenAI(api_key=openai_api_key)

In [ ]:
context = build_final_context(chunks)
message = [{"role": "user", "content": question.format(context=context)}]

response = llm.chat.completions.create(model='gpt-3.5-turbo',
      messages= message,
      temperature=0.1,
      max_tokens=4096)

In [ ]:
response.choices[0].message.content

"VinaLLaMA is an open-source Large Language Model for the Vietnamese language, built upon LLaMA-2 with an additional 800 billion trained tokens. It demonstrates fluency in Vietnamese and understanding of Vietnamese culture. VinaLLaMA-7B-chat achieves SOTA results on key benchmarks, marking a significant advancement in the Vietnamese AI landscape. The model is accessible on HuggingFace, contributing to global AI research and enhancing Vietnamese language processing. The training procedure involves pretraining on public and synthetic data, supervised instruction fine-tuning, and model evaluation on VLSP, VMLU, and Vicuna Benchmark Vietnamese. VinaLLaMA showcases commendable performance in Vietnamese language tasks, with potential for further enhancements. The model's success is attributed to strategic use of synthetic data in training, highlighting the importance of diverse and well-designed datasets in developing robust language models."

context=build_final_context(chunks)

In [ ]:
summarize_chunks = []
for chunk in chunks:
  message = [{"role": "user", "content": summarize_prompt.format(context=chunk)}]
  response = llm.chat.completions.create(model='gpt-3.5-turbo',
        messages= message,
        temperature=0.1,
        max_tokens=4096)
  summarize_chunks.append(response.choices[0].message.content)

conext = build_final_context(summarize_chunks)
message = [{"role": "user", "content": final_ans_prompt.format(context=context)}]

final_response = llm.chat.completions.create(model='gpt-3.5-turbo',
      messages= message,
      temperature=0.1,
      max_tokens=4096)



In [ ]:
final_response.choices[0].message.content

'VinaLLaMA là một Large Language Model tiên tiến cho ngôn ngữ Việt Nam, được xây dựng trên nền tảng LLaMA-2 với 800 tỷ token được huấn luyện thêm. Model này không chỉ thể hiện sự trôi chảy trong tiếng Việt mà còn hiểu sâu về văn hóa Việt Nam. VinaLLaMA-7B-chat đạt kết quả SOTA trên các bài kiểm tra quan trọng, đánh dấu một bước tiến quan trọng trong lĩnh vực AI tiếng Việt. Qua việc sử dụng dữ liệu tổng hợp chất lượng cao, model này đã chứng minh khả năng vượt trội và đa dạng ứng dụng trong nghiên cứu NLP.'

## Query Refinement

In [ ]:
samples_retrieval = ['Hôm nay, giá Bitcoin đã đạt đến một đỉnh mới.',
           'Phở Việt Nam rất ngon, Hà Nội nổi tiếng với món Phở Bò.',
           'Phở là món ăn Việt Nam xuất hiện ở rất nhiều nơi trên thế giới.',
           'Một thị trường khó như Nhật cũng rất ấn tượng với món Phở Bò Việt Nam.']

context = build_final_context(samples_retrieval)
print(context)

Context 1: Hôm nay, giá Bitcoin đã đạt đến một đỉnh mới.
Context 2: Phở Việt Nam rất ngon, Hà Nội nổi tiếng với món Phở Bò.
Context 3: Phở là món ăn Việt Nam xuất hiện ở rất nhiều nơi trên thế giới.
Context 4: Một thị trường khó như Nhật cũng rất ấn tượng với món Phở Bò Việt Nam.



In [ ]:
sample_question = "Hãy giới thiệu về món Phở Bò Việt Nam"
refinement_prompt = """
Dưới đây là các nội dung được trích xuất từ Database:
---CONTEXT---
{context}
---END CONTEXT---
Nhiệm vụ của bạn là hãy viết lại những nội dung trên dựa trên câu hỏi dưới đây, hãy loại bỏ các nội dung không liên quan để tránh gây nhiễu loạn. Tôi chỉ cần phần viết lại của bạn, vui lòng không thêm bất kì thông tin nào khác.
Câu hỏi: {question}
"""

sample_message = [{"role": "user", "content": refinement_prompt.format(context=context, question=sample_question)}]

sample_response = llm.chat.completions.create(model='gpt-3.5-turbo',
      messages=sample_message,
      temperature=0.1,
      max_tokens=4096)

In [ ]:
print(sample_response.choices[0].message.content)

Phở Bò Việt Nam là một món ăn ngon và nổi tiếng của Việt Nam, được biết đến với hương vị đậm đà và thơm ngon. Món ăn này xuất hiện ở nhiều nơi trên thế giới và được đánh giá cao, ngay cả ở những thị trường khó tính như Nhật Bản.


In [ ]:
answer_prompt = """
Dưới đây là các nội dung được trích xuất từ Database:
---CONTEXT---
{context}
---END CONTEXT---
Nhiệm vụ của bạn là trả lời câu hỏi dựa trên các nội dung trên. Tôi chỉ cần trả lời câu hỏi, vui lòng không thêm bất kì thông tin nào khác.
Nếu thư thông tin đưa cho không liên quan đến câu hỏi, vui lòng trả lời "Tôi không biết"
Câu hỏi: {question}
Trả lời:
"""

step2_message = [{"role": "user", "content": answer_prompt.format(context=context, question=sample_question)}]

step2_response = llm.chat.completions.create(model='gpt-3.5-turbo',
      messages=step2_message,
      temperature=0.1,
      max_tokens=4096)

print(step2_response.choices[0].message.content)

Phở Bò là một món ăn truyền thống của Việt Nam, nổi tiếng với hương vị đậm đà và thơm ngon. Món ăn này được làm từ nước dùng phở, thịt bò, bún, và các loại gia vị như hành, gừng, và hồ tiêu. Phở Bò thường được ăn kèm với rau sống, giá, và chanh để tạo thêm hương vị đặc trưng. Món Phở Bò đã trở thành biểu tượng của ẩm thực Việt Nam và được yêu thích trên toàn thế giới.
